#### 0. Pre-Ingestion

In [14]:
# Importing required libraries
import os
import subprocess
import time
import logging
from typing import Literal
from collections import defaultdict

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import wkt
import fiona
import matplotlib.pyplot as plt
import seaborn as sns

# Import utility constants and functions
import utils

In [3]:
# Setup logging
logging.basicConfig(
    level=logging.INFO,  # or DEBUG
    format='%(asctime)s - %(levelname)s - %(message)s',
)
logger = logging.getLogger(__name__)

In [4]:
# Setup logger with time capture
from functools import wraps

def log_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        logger.info(f"Starting: {func.__name__}")
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        logger.info(f"Completed: {func.__name__} in {end - start:.2f} seconds")
        return result
    return wrapper

In [5]:
# Define constants here
geo_crs = "EPSG:4326"
projected_crs = "EPSG:3857" 
POC_FINALIZED_COUNTIES = [
    # urban
    '17031',
    '13121',
    '53033',
    # sub-urban
    '48491',
    '29181',
    '42011',
    # rural 
    '55107',
    '35051',
    '17127'
]

ENCUMBRANCES = [
    'roadways',
    'railways',
    'protected_lands',
    'wetlands',
    'transmission_lines',
    ]
EncumbranceType = Literal[
    'roadways',
    'railways',
    'protected_lands',
    'wetlands',
    'transmission_lines',
]

# Parquet formats are available
# Files are saved as {fips_encumbrance.parquet} or {fips_parcels.parquet} 
PARQUET_FOLDER = r"C:\Users\eprashar\OneDrive - CoreLogic Solutions, LLC\github\jan_25_proj_infra_parcels\data\ingestion_parquets"

#### 1. Ingestion: Encumbrance and Parcel data from pre-processed parquet files

In [6]:
# Define function to get encumbrance parquet for the county
def load_encumbrance_data(
        fips_code:str,
        encumbrance:str):
    """
    """
    # Load encumbrance data saved in local
    gdf_encumbrance = gpd.read_parquet(os.path.join(PARQUET_FOLDER, f"{fips_code}_{encumbrance}.parquet"))
    
    # Convert to EPSG:4326
    gdf_encumbrance = gdf_encumbrance.to_crs(geo_crs)
    print(f'CRS of the {encumbrance} dataframe is {gdf_encumbrance.crs}')
    return gdf_encumbrance

In [7]:
# Define function to get parcel data for the defined county
def load_parcel_data(fips_code: str) -> gpd.GeoDataFrame:
    """
    Load parcel data from BigQuery and filter by FIPS code.
    """
    # Load parcel parquet file saved in local
    gdf_parcel = gpd.read_parquet(os.path.join(PARQUET_FOLDER, f"{fips_code}_parcels.parquet"))
    
    # Convert to EPSG:4326
    gdf_parcel = gdf_parcel.to_crs(geo_crs)
    print(f'CRS of the parcel dataframe is {gdf_parcel.crs}')
    return gdf_parcel

#### 2. Calculating Proximity and Intersection Metrics
*Intersecting all parcels with buffered values of encumbrance*

In [8]:
# Define buffer distances and scores based on polygon or line geometry
def buffer_scores_and_labels(
        encumbrance: EncumbranceType):
    '''
    Define buffer distances and score labels based on encumbrance type.
    '''
    if encumbrance == 'railways' or encumbrance == 'transmission_lines':
        buffer_distances = [5, 150, 300, 750, 1000]
        score_labels = ['intersects', 'very high', 'high', 'medium', 'low']
    
    elif encumbrance == 'roadways':
        buffer_distances = [5, 10, 25, 50, 100]
        score_labels = ['intersects', 'very high', 'high', 'medium', 'low']
    
    elif encumbrance == 'wetlands' or encumbrance == 'protected_lands':
        buffer_distances = [0, 5, 10, 50, 100]
        score_labels = ['intersects', 'very high', 'high', 'medium', 'low']

    return buffer_distances, score_labels

In [9]:
# Function to calculate intersection metrics
def calculate_intersection_metrics(
        encumbrance:EncumbranceType,
        all_parcels,
        matched_parcels,
        buffered_encumbrance
):
    '''

    '''
    # Reset index to make sure merge works properly
    buffer_gdf = buffered_encumbrance.reset_index().rename(columns={'index': 'buffer_index'})

    # Merge buffer geometries into matched results using index_right from sjoin
    matched_with_geom = matched_parcels.reset_index().merge(
        buffer_gdf[['buffer_index', 'geometry']],
        left_on='index_right',
        right_on='buffer_index',
        how='left',
        suffixes=('', '_buffer')
    )
    # Ensure both geometries are in a projected CRS (3857) for accurate area
    # Setting projection for parcel geometry
    matched_with_geom = matched_with_geom.set_geometry('geometry')
    matched_with_geom = matched_with_geom.to_crs(projected_crs)
    
    # Setting projection for buffered value of encumbrance geometry 
    matched_with_geom = matched_with_geom.set_geometry('geometry_buffer', drop=False)
    matched_with_geom[f'geometry_buffer_{encumbrance}'] = matched_with_geom['geometry_buffer'].to_crs(projected_crs)

    # Calculate intersection geometry 
    matched_with_geom[f'intersection_geom_{encumbrance}'] = matched_with_geom['geometry'].intersection(matched_with_geom[f'geometry_buffer_{encumbrance}'])

    # Group by parcel index -- this will help later to pick parcel row in case of multiple intersections
    matched_with_geom['orig_index'] = matched_with_geom['index']  # preserve original index for reassigning

    # Calculate intersection metrics for lines
    if encumbrance in ['railways', 'roadways', 'transmission_lines']:
        
        # Calculate intersection perimeter
        matched_with_geom[f'intersec_perim_{encumbrance}'] = matched_with_geom[f'intersection_geom_{encumbrance}'].length

        # Approximate true line length (perimeter / 2)
        matched_with_geom[f'approx_line_len_{encumbrance}'] = round(
            matched_with_geom[f'intersec_perim_{encumbrance}'] / 2, 2
        )

        # Retain row with max approximate line length
        max_length_idx = matched_with_geom.groupby('orig_index')[f'approx_line_len_{encumbrance}'].idxmax()
        matched_max_area = matched_with_geom.loc[max_length_idx]

        # Store this back into parcels dataframe
        all_parcels.loc[matched_max_area['orig_index'], f'approx_line_len_{encumbrance}'] = matched_max_area[f'approx_line_len_{encumbrance}'].values

    elif encumbrance in ['wetlands', 'protected_lands']:

        # Calculate parcel intersection ratio
        matched_with_geom[f'intersec_area_{encumbrance}'] = round(matched_with_geom[f'intersection_geom_{encumbrance}'].area,2)
        matched_with_geom['parcel_area'] = matched_with_geom['geometry'].area
        matched_with_geom[f'area_ratio_{encumbrance}'] = round((
            matched_with_geom[f'intersec_area_{encumbrance}'] / matched_with_geom['parcel_area']
        ),2)
        
        # Calculate parcel centroid to wetland distance
        # Convert parcel centroid column to GeoSeries
        # Convert WKT strings to actual shapely geometries
        matched_with_geom['centroid'] = matched_with_geom['centroid'].apply(wkt.loads)
        matched_with_geom['centroid'] = gpd.GeoSeries(matched_with_geom['centroid'], crs=geo_crs)
        
        # Reproject centroid to projected CRS (e.g. 3857)
        matched_with_geom['centroid'] = matched_with_geom['centroid'].to_crs(projected_crs)

        # parcel centroid to encumbered geometry distance
        matched_with_geom[f'parcel_dist_to_{encumbrance}'] = round(matched_with_geom['centroid'].distance(
            matched_with_geom[f'geometry_buffer_{encumbrance}']
            ),2)
        
        # Chunk to retain row with the max area ratio
        max_area_idx = matched_with_geom.groupby('orig_index')[f'area_ratio_{encumbrance}'].idxmax()
        matched_max_area = matched_with_geom.loc[max_area_idx]

        # Store metrics back in parcels_mod for max intersection row
        for col in ['intersec_area', 'area_ratio', 'parcel_dist_to']:
            all_parcels.loc[matched_max_area['orig_index'], f'{col}_{encumbrance}'] = matched_max_area[f'{col}_{encumbrance}'].values

    # Calculate number of intersecting encumbrances per parcel (for all encumbrance types)
    intersection_counts = matched_with_geom.groupby('orig_index').size()
    
    # Store the number of intersections as a separate metric
    all_parcels.loc[intersection_counts.index, f'n_{encumbrance}_intersections'] = intersection_counts.values
    
    logger.info(f"Finished calculating intersection metrics for encumbrance {encumbrance}!")
    return all_parcels

In [ ]:
# Calculate proximity score and intersection metrics based on encumbrance type
@log_time
def get_proximity_score_and_intersection_metrics(
        encumbrance: EncumbranceType,
        gdf_parcel: gpd.GeoDataFrame, 
        gdf_encumbrance: gpd.GeoDataFrame, 
        ) -> gpd.GeoDataFrame:
    """
    Assign proximity scores to parcels based on their distance to encumbrance features.
    
    Parameters:
    parcels (GeoDataFrame): The parcels to be scored.
    encumbrance (GeoDataFrame): The encumbrance features to score against.
    
    Returns:
    GeoDataFrame: Parcels with assigned proximity scores.
    """
    # Start logging process
    logger.info("Starting proximity scoring...")

    # Derive scores based on nature of encumbrance
    buffer_distances, score_labels = buffer_scores_and_labels(encumbrance)
    logger.info(f"Buffer distances: {buffer_distances}")
    logger.info(f"Score labels: {score_labels}")
    
    # Ensure the CRS of both GeoDataFrames match
    gdf_encumbrance = gdf_encumbrance.to_crs(gdf_parcel.crs)

    # Create a copy of the parcels GeoDataFrame to avoid modifying the original
    parcels_mod = gdf_parcel.copy()

    # Initialize a new column for proximity score
    # Explicitly defining dtype object to avoid SettingWithCopyWarning
    parcels_mod[f'proximity_score_{encumbrance}'] = pd.Series([None]*len(parcels_mod), dtype='object')

    # Calculate proximity scores based on distance to encumbrance features
    # Initialize i 
    i = 0
    for distance, label in zip(buffer_distances, score_labels):
        
        # Buffer individual geometries by specified distance
        # Project to a projected CRS for buffering
        buffer_gdf = gdf_encumbrance.to_crs(projected_crs)

        # Create buffered geometries and assign to a new column
        buffer_gdf['buffered_geometry'] = buffer_gdf.geometry.buffer(distance) # This geometry holds the buffered version

        # Drop the original geometry column
        buffer_gdf = buffer_gdf.drop(columns=['geometry'])

        # Rename the buffered geometry column to 'geometry'
        buffer_gdf = buffer_gdf.rename(columns={'buffered_geometry': 'geometry'})

        # Set CRS and reproject back to geo_crs
        buffer_gdf = buffer_gdf.set_geometry('geometry').to_crs(geo_crs)
        logger.info(f'Created buffered geometry with distance {distance} meters and CRS {buffer_gdf.crs}')
        
        # Use spatial join to find parcels within the buffer distance
        matched = gpd.sjoin(parcels_mod[parcels_mod[f'proximity_score_{encumbrance}'].isna()],
                             buffer_gdf,
                             predicate='intersects',
                             how='inner')

        # Assign labels to matched parcels based on proximity score
        # Use the index of matched parcels to assign the proximity score
        parcels_mod.loc[matched.index, f'proximity_score_{encumbrance}'] = label
        
        # Add encumbrance column values to main dataframe
        # TODO: Find more elegant solution
        cols_to_add = buffer_gdf.columns.difference(['geometry'])
        for col in cols_to_add:
            parcels_mod.loc[matched.index, f"{col.lower()}_{encumbrance}"] = matched[col].values
        logger.info(f"Assigned proximity score {label} to {len(matched)} parcels...")

        # Add intersecton metrics when buffer is smallest
        if i == 0:
            logger.info('Now adding intersection metrics...')
            parcels_mod = calculate_intersection_metrics(
                encumbrance=encumbrance,
                all_parcels=parcels_mod,
                matched_parcels=matched,
                buffered_encumbrance=buffer_gdf
            )
        # Increment i
        i += 1 
            
    # Fill remaining as no encumbrance and change to geographic CRS
    parcels_mod.fillna({f'proximity_score_{encumbrance}':'no_encumbrance'}, inplace=True)
    
    # Re-project everything to geographic CRS
    for column in parcels_mod.select_dtypes(include=['geometry']).columns:
        parcels_mod[column] = parcels_mod[column].to_crs(geo_crs)
    print(f'CRS of output dataframe is {parcels_mod.crs}')
    print('Proximity scoring complete! Counts of proximity scores are...')

    # Print value counts of proximity scores
    print(parcels_mod[f'proximity_score_{encumbrance}'].value_counts())
    return parcels_mod

#### 3. Calculating Intersection Strength (For Polygons)

In [ ]:
# Function to calculate intersection strength score
def calculate_intersection_score(
        encumbrance:EncumbranceType,
        gdf_parcel: gpd.GeoDataFrame,
        *,
        area_ratio_weight: float = 0.5,
        dist_weight: float = 0.3,
        n_intersections_weight: float = 0.2,
        area_ratio_thresholds: tuple = (0.4, 0.9),
        dist_thresholds: tuple = (100, 50, 10, 0),
        n_intersections_thresholds: tuple = (2 ,3),
        score_thresholds: tuple = (0.35, 0.7)
    ) -> gpd.GeoDataFrame:
    '''
    Calculates intersection strength score for a given encumbrance using:
    - Area ratio
    - Centroid distance
    - Number of intersections
    
    Accepts tunable weights and thresholds via kwargs.

    Returns:
    GeoDataFrame with a new 'intersection_score_{encumbrance}' column.
    '''
    # Check score is only asked for polygon encumbrances
    if encumbrance not in ['wetlands', 'protected_lands']:
        raise ValueError(
            f"Intersection scoring is only applicable for polygon encumbrances. "
            f"Valid options are: 'wetlands', 'protected_lands'."
        )

    # Create dataframe copy to avoid modifying the original
    parcels_mod = gdf_parcel.copy()

    # Unpack thresholds
    ar_low, ar_high = area_ratio_thresholds
    dist_low, dist_med, dist_high, dist_overwrite = dist_thresholds
    nint_med, nint_high = n_intersections_thresholds

    # Score area_ratio
    ar_col = f'area_ratio_{encumbrance}'
    parcels_mod[f'score_ar_{encumbrance}'] = parcels_mod[ar_col].apply(
        lambda x: 1 if x >= ar_high else (0.5 if x >= ar_low else 0.25 if x > 0 else 0)
    )

    # Score parcel_dist_to
    dist_col = f'parcel_dist_to_{encumbrance}'
    parcels_mod[f'score_dist_{encumbrance}'] = parcels_mod[dist_col].apply(
        lambda x: 1 if x <= dist_high else (0.5 if x <= dist_med else 0.25 if x <= dist_low else 0.15 if x > 0 else 0)
    )

    # Score number of intersections
    nint_col = f'n_{encumbrance}_intersections'
    parcels_mod[f'score_nint_{encumbrance}'] = parcels_mod[nint_col].apply(
        lambda x: 1 if x >= nint_high else (0.5 if x == nint_med else 0.25 if x > 0 else 0)
    )

    # Final weighted score
    score_col = f'intersection_score_{encumbrance}'
    parcels_mod[score_col] = (
        parcels_mod[f'score_ar_{encumbrance}'] * area_ratio_weight +
        parcels_mod[f'score_dist_{encumbrance}'] * dist_weight +
        parcels_mod[f'score_nint_{encumbrance}'] * n_intersections_weight
    )

    # TODO: Once testing is over, drop intermediate columns
    # parcels_mod = parcels_mod.drop(columns=[
    # f'parcel_dist_to_{encumbrance}',
    # f'area_ratio_{encumbrance}',
    # f'n_{encumbrance}_intersections'])

    # Add low, medium, high labels
    # Labeling with override for high-impact flags
    # TODO: Make this more efficient
    low_thres, high_thres = score_thresholds
    label_col = f'intersection_label_{encumbrance}'

    parcels_mod[label_col] = parcels_mod.apply(
    lambda row: None if row[score_col] == 0 else (
        'high' if (
            row[ar_col] >= ar_high 
            or row[dist_col] == dist_overwrite
        ) else (
            'low' if row[score_col] < low_thres else
            'medium' if row[score_col] < high_thres else
            'high'
        )
    ),
    axis=1
)

    print(f'Intersection scoring completed for {encumbrance}!')
    return parcels_mod

#### 4. Calculating Summary Stats Using Calculated Metrics
**Questions from a Product perspective:**
1. What are the top x land uses for parcels in a given proximity bucket?
2. Who are the top x owners in a given proximity bucket?

3. What do the total parcel value quartiles look like across proximity buckets?
4. What is the quartile values for sq. footage across proximity buckets?
5. What are quartile values for value/sq.ft. across proximity buckets?
6. What % of parcels in a given proximity bucket are taxable?
7. For taxable parcels, what are the quartile values for tax amount?


In [12]:
# Function to plot quartile values for a given measure and proximity labels
def plot_measure_quartiles(
        df, 
        grouping_col, 
        labels = ['low', 'medium', 'high'],
        measures= ['tot_val', 'land_sq_ft', 'val_per_sq_ft','tax_amt', 'tax_per_sq_ft'],
        quantiles = [0.25, 0.50, 0.75, 0.90, 0.95]
        ):
    """
    Plots Q25, Q50, Q75 lines for each measure and proximity label.

    Parameters:
        df (pd.DataFrame): Raw parcel data with proximity_score and measure columns.
        labels (list of str): e.g., ['high', 'medium']
        measures (list of str): e.g., ['tot_val', 'land_sq_ft']
    """
    # Filter for selected proximity labels
    df_new = df[df[grouping_col].isin(labels)]

    for measure in measures:

        # Copying dataframe to avoid warnings
        df_filtered = df_new.copy()
        
        if measure == 'val_per_sq_ft':

            # Create a mask for valid values
            valid_mask = df_filtered['tot_val'].notna() & df_filtered['land_sq_ft'].notna() & (df_filtered['land_sq_ft'] != 0)

            # Create the new column safely
            df_filtered.loc[valid_mask, 'val_per_sq_ft'] = df_filtered.loc[valid_mask, 'tot_val'] / df_filtered.loc[valid_mask, 'land_sq_ft']

            # Round to two decimal points
            df_filtered['val_per_sq_ft'] = df_filtered['val_per_sq_ft'].round(2)
        
        elif measure == 'tax_per_sq_ft':
            # Create a mask for valid values
            valid_mask = df_filtered['tax_amt'].notna() & df_filtered['land_sq_ft'].notna() & (df_filtered['land_sq_ft'] != 0)

            # Create the new column safely
            df_filtered.loc[valid_mask, 'tax_per_sq_ft'] = df_filtered.loc[valid_mask, 'tax_amt'] / df_filtered.loc[valid_mask, 'land_sq_ft']

            # Round to two decimal points
            df_filtered['tax_per_sq_ft'] = df_filtered['tax_per_sq_ft'].round(2)


        plt.figure(figsize=(10,4))

        for label in labels:
            group = df_filtered[df_filtered[grouping_col] == label]

            # Drop missing values for the measure
            values = group[measure].dropna()

            if values.empty:
                continue

            # Compute quantiles dynamically
            q_vals = values.quantile(quantiles).values
            q_labels = [f"Q{int(q*100)}" for q in quantiles]

            # Plot the values
            plt.plot(q_labels, q_vals, marker='o', label=label)

        plt.title(f"Quantile Trends for {measure.replace('_', ' ').title()}")
        plt.xlabel("Quantile")
        plt.ylabel(measure.replace('_', ' ').title())
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.legend(title=f"{grouping_col} Labels")
        plt.tight_layout()
        plt.show()

In [13]:
# Visualization to display top x land_use codes/owners for a given proximity bucket
def plot_top_x(
        df, 
        grouping_col,
        proximity_label,
        use_case='land_use',
        top_x=10):
    """
    Plots top X land use codes for a given proximity bucket.
    
    Parameters:
        df (pd.DataFrame): DataFrame with columns ['proximity_score', 'land_use_t', 'count']
        proximity_label (str): e.g., 'high', 'medium', 'low', 'no_encumbrance', etc.
        use_case should ideally be either land_use_t or owner
        top_x (int): number of top land use codes to show
    """
    # Filter the relevant proximity bucket
    subset = df[df[grouping_col] == proximity_label]

    if subset.empty:
        print(f"No data found for proximity score: {proximity_label}")
        return
    
    if use_case == 'land_use':
        aggregation = 'land_use_t'
    elif use_case == 'owner':
        aggregation = 'owner'
    
    # Get land use counts
    land_use_counts = (
    subset.groupby(aggregation)
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
    )

    # Get top X land uses
    top_land_uses = land_use_counts.nlargest(top_x, 'count')

    # Plot
    plt.figure(figsize=(8, 6))
    sns.barplot(
        data=top_land_uses,
        x='count',
        y=aggregation,
        #palette='viridis'
    )
    plt.title(f"Top {top_x} Land Use Types for Proximity: {proximity_label}")
    plt.xlabel("Parcel Count")
    plt.ylabel("Land Use Type")
    plt.grid(True, axis='x', linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()